In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import seaborn as sns
import fastf1
from src.plotset import setup_plot
from fastf1 import plotting

setup_plot()

In [ ]:
fastf1.Cache.enable_cache('./f1_cache')
fastf1.Cache.get_cache_info()

In [ ]:
session = fastf1.get_session(2025,14,'Q')
session.load()

In [ ]:
q1, q2, q3 = session.laps.split_qualifying_sessions()

In [ ]:
df = session.results.iloc[:10][['Q1','Q2','Q3']].copy()
df['Q1'] = df['Q1'].dt.total_seconds()
df['Q2'] = df['Q2'].dt.total_seconds()
df['Q3'] = df['Q3'].dt.total_seconds()

In [ ]:
# df = pd.DataFrame({'Q1':q1.pick_quicklaps().LapTime.dt.total_seconds(),
#                    'Q2':q2.pick_quicklaps().LapTime.dt.total_seconds(),
#                    'Q3':q3.pick_quicklaps().LapTime.dt.total_seconds()})

In [ ]:
setup_plot(xyticksize=20,axeslabel=22,figtitle=24,)
fig, ax = plt.subplots(figsize=(15,8))
sns.violinplot(data=df,inner=None,palette='Set1',width=0.5,ax=ax)
sns.swarmplot(data=df,size=20,linewidth=3,edgecolor='k',palette='Set1',ax=ax)

ax.set_title('(Top 10) Lap Time Evolution: Q1 --> Q3')
ax.set_ylabel('Lap Times (sec)')
ax.set_ylim(74.5, 76.25)

In [ ]:
df.Q1.mean(), df.Q2.mean(),df.Q3.mean()

In [ ]:
fig.savefig('track_evolution.png',dpi=300, bbox_inches='tight')

In [ ]:
lec_color = plotting.get_driver_color(session=session,identifier='LEC')
pia_color = plotting.get_driver_color(session=session,identifier='PIA')

In [ ]:
lec_df = session.laps.pick_drivers('LEC').pick_fastest().telemetry[['Time','Distance','X','Y','Speed']].copy()
pia_df = session.laps.pick_drivers('PIA').pick_fastest().telemetry[['Time','Distance','X','Y','Speed']].copy()

In [ ]:
lec_df['Seconds'] = lec_df['Time'].dt.total_seconds()
pia_df['Seconds'] = pia_df['Time'].dt.total_seconds()

In [ ]:
min_dist = max(pia_df['Distance'].min(), lec_df['Distance'].min())
max_dist = min(pia_df['Distance'].max(), lec_df['Distance'].max())

distance_grid = np.linspace(min_dist, max_dist, num=1000)

In [ ]:
# Interpolated cumulative time
lec_time = np.interp(distance_grid, lec_df['Distance'], lec_df['Seconds'])
pia_time = np.interp(distance_grid, pia_df['Distance'], pia_df['Seconds'])

# X, Y positions
lec_x = np.interp(distance_grid, lec_df['Distance'], lec_df['X'])
lec_y = np.interp(distance_grid, lec_df['Distance'], lec_df['Y'])
pia_x = np.interp(distance_grid, pia_df['Distance'], pia_df['X'])
pia_y = np.interp(distance_grid, pia_df['Distance'], pia_df['Y'])

# Speed
lec_speed = np.interp(distance_grid, lec_df['Distance'], lec_df['Speed'])
pia_speed = np.interp(distance_grid, pia_df['Distance'], pia_df['Speed'])

In [ ]:
delta = pia_speed - lec_speed  # Negative -> Piastri faster

# Assign colors
colors = np.where(delta > 0, pia_color, lec_color)

In [ ]:
# Interpolated positions
x = (lec_x + pia_x) / 2
y = (lec_y + pia_y) / 2

theta = np.deg2rad(39.5)  # Convert 45° to radians
x_rot = x * np.cos(theta) - y * np.sin(theta)
y_rot = x * np.sin(theta) + y * np.cos(theta)

In [ ]:
points = np.array([x_rot, y_rot]).T.reshape(-1,1,2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

In [ ]:
import matplotlib.patches as mpatches

# Create legend handles manually
pia_patch = mpatches.Patch(color=pia_color, label='Piastri faster')
lec_patch = mpatches.Patch(color=lec_color, label='Leclerc faster')

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
fig.set_facecolor('#000000')
ax.axis('off')

# Color per segment (drop last color to match segments length)
lc = LineCollection(segments, colors=colors[:-1], linewidth=16)

ax.add_collection(lc)
ax.autoscale()

# Add legend to your axes
ax.legend(handles=[lec_patch, pia_patch],
          loc='best',  # or any location you prefer
          bbox_to_anchor=(1,1),
          frameon=True,
          facecolor='k',
          fontsize=20,
          labelcolor='white')

plt.tight_layout()
plt.show()

In [ ]:
fig.savefig('track_dominance.png',dpi=300, bbox_inches='tight')